In [1]:
from fuzzywuzzy import fuzz
import spacy

In [2]:
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


d:\PROJECTS\Service for Elasticsearch\araks_venv\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
async def fuzzy_similar(a, b):
    return (fuzz.token_sort_ratio(a, b) + fuzz.partial_token_set_ratio(a, b)) / 2

In [4]:
async def exact_similar(a, b):
    return (sorted(a.lower().split()) == sorted(b.lower().split()))

In [5]:
source_props = ["Danoyan", "Yerevan 15 Armenia"]
target_props = ["15 Yerevan Armenia", "Armen Danoyan"]

In [6]:
nlp(source_props[0]).similarity(nlp(target_props[1]))

0.750638044548437

In [69]:
async def match(source, target, method):
    if method == "fuzzy":
                ratio = await fuzzy_similar(source['name'], target['name'])
                if ratio > 80:
                    return source['id'], target['id']
                    
    elif method == "exact":
                ratio = await exact_similar(source['name'], target['name'])
                if ratio:
                    return source['id'], target['id']

In [93]:
data = [{"source" : [{"name" : "Yerevan 15 Armenia", "id" : 0}], "target" : [{"name" : "15 Yerevan Armenia", "id" : 1}], "method" : "fuzzy"}, {"source" : [{"name" : "Danoyan Armen", "id" : 2}], "target" : [{"name" : "Armen   Danoyan", "id" : 3}], "method" : "exact"}]

In [94]:
import asyncio

async def process_relations(rel_data):
    
    matched_ids = []
    
    for rel in rel_data:
        sources = rel["source"]
        targets = rel["target"]
        method = rel["method"]
        
        tasks = [match(source, target, method) for source in sources for target in targets]
        results = await asyncio.gather(*tasks)
        
        matched_ids.extend(results)
    
    return matched_ids

In [95]:
await process_relations(data)

[(0, 1), (2, 3)]